<a href="https://colab.research.google.com/github/hou1020/CASA0025/blob/main/W05__quiz_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1

This URL: https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip points to a.zip file containing shapefiles associated with
the NYC data that we've been using in class so far. You can choose how to use sql,
either the con.sql() syntax or the %%sql syntax. Either way, use the code cell
below to install and import the necessary packages.

In [1]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.8/667.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 14.6 M

In [3]:
url = "https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip "
leafmap.download_file(url, unzip=True)

Downloading...
From: https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip 
To: /content/postgis-workshop-2020.zip 
297B [00:00, 810kB/s]


'/content/postgis-workshop-2020.zip '

In [4]:
con = duckdb.connect("data.db")
con.install_extension("spatial")
con.load_extension("spatial")

Q1

Firslty, please provide the link to your colab notebook in the Moodle quiz using the
following steps:

In the top right corner, click the "share" button.

In the popup, click on "restricted" and change this to "anyone with the link"

Finally, click "copy link", and paste the link into the box for Question 1 on moodle.

Once you've done this, download and unzip the data, then create the following tables using the corresponding shapefiles.

nyc_neighborhoods

nyc_census_blocks

nyc_homicides

nyc_streets

nyc_subway_stations

In [5]:
con.sql("CREATE OR REPLACE TABLE nyc_census_blocks AS SELECT * FROM ST_Read('nyc_census_blocks.shp')")
con.sql("CREATE OR REPLACE TABLE nyc_homicides AS SELECT * FROM ST_Read('nyc_homicides.shp')")
con.sql("CREATE OR REPLACE TABLE nyc_neighborhoods AS SELECT * FROM ST_Read('nyc_neighborhoods.shp')")
con.sql("CREATE OR REPLACE TABLE nyc_streets AS SELECT * FROM ST_Read('nyc_streets.shp')")
con.sql("CREATE OR REPLACE TABLE nyc_subway_stations AS SELECT * FROM ST_Read('nyc_subway_stations.shp')")

In [6]:
con.sql("SELECT * FROM nyc_census_blocks LIMIT 5").show()
con.sql("SELECT * FROM nyc_homicides LIMIT 5").show()
con.sql("SELECT * FROM nyc_neighborhoods LIMIT 5").show()
con.sql("SELECT * FROM nyc_streets LIMIT 5").show()
con.sql("SELECT * FROM nyc_subway_stations LIMIT 5").show()

┌─────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│      BLKID      │ POPN_TOTAL │ POPN_WHITE │ POPN_BLACK │ POPN_NATIV │ POPN_ASIAN │ POPN_OTHER │   BORONAME    │                      

Q2 What is the longest street in New York? Note, it may be split up into multiple
segments! Ignore missing and null values.

In [15]:
con.sql("""
    SELECT name, sum(ST_Length(geom)) AS length
    FROM nyc_streets
    GROUP BY name
    ORDER BY length DESC
    LIMIT 2
""")

┌───────────────────┬────────────────────┐
│       NAME        │       length       │
│      varchar      │       double       │
├───────────────────┼────────────────────┤
│ NULL              │  531715.4000027523 │
│ Grand Central Pky │ 40210.484038252165 │
└───────────────────┴────────────────────┘

Q3 Which borough had the fewest daytime shootings in 2009?

In [23]:
con.sql("""
SELECT BORONAME, COUNT(*) AS count
FROM nyc_homicides
WHERE YEAR=2009 AND LIGHT_DARK='L'
GROUP BY BORONAME
ORDER BY count ASC
""")

┌───────────────┬───────┐
│   BORONAME    │ count │
│    varchar    │ int64 │
├───────────────┼───────┤
│ Staten Island │     4 │
│ Manhattan     │    12 │
│ The Bronx     │    17 │
│ Queens        │    26 │
│ Brooklyn      │    44 │
└───────────────┴───────┘

Q4 What is the total population of the census blocks served by the L train?

In [35]:
con.sql("""
SELECT sum(c.POPN_TOTAL)
FROM nyc_census_blocks AS c
JOIN nyc_subway_stations AS s
ON ST_Intersects(c.geom, s.geom)
WHERE s.routes LIKE '%L%'
""")

┌───────────────────┐
│ sum(c.POPN_TOTAL) │
│      int128       │
├───────────────────┤
│              7147 │
└───────────────────┘

Q5 which subway station had the largest number of homicides within 1km?

In [39]:
con.sql("""
SELECT s.name, COUNT(*) AS count
FROM nyc_homicides AS h
JOIN nyc_subway_stations AS s
ON ST_DWithin(h.geom, s.geom, 1000)
GROUP BY s.name
ORDER BY count DESC
LIMIT 1
""")

┌──────────┬───────┐
│   NAME   │ count │
│ varchar  │ int64 │
├──────────┼───────┤
│ 125th St │   305 │
└──────────┴───────┘

# Section 2

This section explores building damage in the Gaza Strip resulting from the ongoing
war. You will conduct a geospatial analysis of building damage using two datasets:
1. Humantiarian Open Street Map building footprints for the Gaza Strip
data: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
2. Damage points from the UN Satellite Agency (UNOSAT).
data: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip

The Coordinate Reference System of these datasets is EPSG:4326; therefore, stock
functions like ST_DISTANCE() will yield values in degrees, not meters. Be mindful of
this in your analysis, and use the DuckDB spatial functions documentation to your
advantage.

First, download and unzip the data, and create two tables: gaza_buildings,
containing the building footprint data, and damage_points containing the
UNOSAT damage points. Make sure to set a spatial index on these tables, this will
make your queries run much faster!

In [40]:
url = "https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip"
leafmap.download_file(url, unzip=True)
url = "https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
To: /content/Gaza_Buildings_2.geojson.zip
100%|██████████| 15.0M/15.0M [00:00<00:00, 83.7MB/s]


Extracting files...


Downloading...
From: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip
To: /content/UNOSAT_GAZA_20240503_2.zip
100%|██████████| 4.98M/4.98M [00:00<00:00, 147MB/s]


Extracting files...


'/content/UNOSAT_GAZA_20240503_2.zip'

In [91]:
con.sql("CREATE OR REPLACE TABLE gaza_buildings AS SELECT * FROM ST_Read('Gaza_Buildings_2.geojson')")
con.sql("CREATE OR REPLACE TABLE damage_points AS SELECT * FROM ST_Read('UNOSAT_GAZA_20240503_2.shp')")
con.sql("CREATE INDEX gaza_buildings_geom_idx ON gaza_buildings USING RTREE(geom)")
con.sql("CREATE INDEX damage_points_geom_idx ON damage_points USING RTREE(geom)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [94]:
con.sql("""
SELECT * FROM gaza_buildings LIMIT 5;
""").show()
con.sql("""
SELECT COUNT(*) FROM gaza_buildings;
""").show()
con.sql("""
SELECT * FROM damage_points LIMIT 5;
""").show()
con.sql("""
SELECT COUNT(*) FROM damage_points;
""").show()

┌────────────┬───────────┬────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│   osm_id   │ osm_type  │  building  │  name   │                                                             geom                                                              │
│   int32    │  varchar  │  varchar   │ varchar │                                                           geometry                                                            │
├────────────┼───────────┼────────────┼─────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 1279368365 │ ways_poly │ yes        │ NULL    │ POLYGON ((34.3603715 31.3771578, 34.360402 31.3771325, 34.3604462 31.3771715, 34.3604158 31.3771967, 34.3603715 31.3771578))  │
│ 1279368369 │ ways_poly │ yes        │ NULL    │ POLYGON ((34.3603239 31.3780182, 34.3603443 31.3780044, 34.3

Q6 Create a table called "gaza_buildings_damaged" which is comprised only of
building footprints that intersect with a UNOSAT damage point. How many
damaged buildings are there in Gaza?

In [92]:
con.sql("""
CREATE OR REPLACE TABLE gaza_buildings_damaged AS
SELECT DISTINCT * FROM gaza_buildings
JOIN damage_points
ON ST_Intersects(gaza_buildings.geom, damage_points.geom);

SELECT COUNT(*) FROM gaza_buildings_damaged;
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       109495 │
└──────────────┘

Q7 What is the total area (in square kilometers) of damaged buildings in Gaza? note:
there are TWO area functions in duckDB, and only one of them returns answers in
meters.

In [52]:
con.sql("""
SELECT SUM(ST_Area_Spheroid(geom))/1000000
FROM gaza_buildings_damaged
""")

┌─────────────────────────────────────────┐
│ (sum(st_area_spheroid(geom)) / 1000000) │
│                 double                  │
├─────────────────────────────────────────┤
│                      22.571649256730982 │
└─────────────────────────────────────────┘

Q8 What percentage of hospitals in Gaza have been damaged?

In [62]:
con.sql("""
SELECT
(SELECT COUNT(*) FROM gaza_buildings_damaged WHERE building LIKE '%hospital%')
/(SELECT COUNT(*) FROM gaza_buildings WHERE building LIKE '%hospital%') AS percentage
""")

┌───────────────────┐
│    percentage     │
│      double       │
├───────────────────┤
│ 0.660377358490566 │
└───────────────────┘

Q9 The UNOSAT data contains a column called "Governorat" (shapefiles only allow
column names of up to 10 characters). Calculate the number of damaged schools
in each governorate.

In [66]:
con.sql("""
SELECT Governorat,COUNT(*) FROM gaza_buildings_damaged
WHERE building LIKE '%school%'
GROUP BY Governorat
""")

┌───────────────┬──────────────┐
│  Governorat   │ count_star() │
│    varchar    │    int64     │
├───────────────┼──────────────┤
│ North Gaza    │           29 │
│ Gaza          │           69 │
│ Deir Al-Balah │            6 │
│ Khan Yunis    │           46 │
└───────────────┴──────────────┘

Q10 The "Dar Al-Shifa Hospital" was the largest medical complex in the gaza strip. Find
the hospital's building footprint, and calculate the proportion of damaged
buildings within 1 kilometer.

In [87]:
con.sql("""
SELECT
    (SELECT COUNT(*)
     FROM gaza_buildings_damaged
     WHERE ST_Distance_Spheroid(
         ST_Centroid(geom),
         (SELECT ST_Centroid(geom) FROM gaza_buildings WHERE name='Dar Al-Shifa Hospital' LIMIT 1)
     ) <= 1000) * 1.0
    /
    (SELECT COUNT(*)
     FROM gaza_buildings
     WHERE ST_Distance_Spheroid(
         ST_Centroid(geom),
         (SELECT ST_Centroid(geom) FROM gaza_buildings WHERE name='Dar Al-Shifa Hospital' LIMIT 1)
     ) <= 1000)
AS proportion
""")

┌────────────────────┐
│     proportion     │
│       double       │
├────────────────────┤
│ 0.5104616980805811 │
└────────────────────┘